In [ ]:
import volstreet.datamodule as dm
import plotly.express as px
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import volstreet as vs
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# Using DataClient class
client = dm.DataClient(api_key=__import__('os').environ['EOD_API_KEY'])
kite_user = __import__('os').environ['KITE_USER']
kite_pass = __import__('os').environ['KITE_PASS']
kite_api_key = __import__('os').environ['KITE_API_KEY']
kite_api_secret = __import__('os').environ['KITE_API_SECRET']
kite_auth_key = __import__('os').environ['KITE_AUTH_KEY']

In [ ]:
# Using get_data and analyser functions
nifty_data = client.get_data(symbol='NIFTY')
bnf_data = client.get_data(symbol='BANKNIFTY')
finnifty_data = client.get_data(symbol='FINNIFTY')
nifty_daily_data = dm.analyser(nifty_data, frequency='D')
bnf_daily_data = dm.analyser(bnf_data, frequency='D')
nifty_weekly_data = dm.analyser(nifty_data, frequency='W-THU')
bnf_weekly_data = dm.analyser(bnf_data, frequency='W-THU')
nifty_monthly_data = dm.analyser(nifty_data, frequency='M-THU')
bnf_monthly_data = dm.analyser(bnf_data, frequency='M-THU')

In [ ]:
# Using ratio_analysis function
rolling_periods = 5
ratio_data = dm.ratio_analysis(bnf_weekly_data, nifty_weekly_data, add_rolling=rolling_periods)
px.line(ratio_data, x=ratio_data.index, y=['BANKNIFTY Change', 'NIFTY Change', f'Rolling {rolling_periods} Ratio'], color_discrete_map={'BANKNIFTY Change': 'red', 'NIFTY Change': 'blue', f'Rolling {rolling_periods} Ratio': 'green'})

In [ ]:
ratio_data

In [ ]:
# Using gambler function for NIFTY and BANKNIFTY
for index in ['NIFTY', 'BANKNIFTY']:
    print(f'{index}\n')
    data = client.get_data(symbol=index)
    for frequency in ['D', 'D-THU', 'W-THU', 'M-THU']:
        print(f'{frequency}\n')
        dm.gambler(data, frequency, 'abs_change')

In [ ]:
# Using gambler function for FINNIFTY
for index in ['FINNIFTY']:
    print(f'{index}\n')
    data = client.get_data(symbol=index)
    for frequency in ['D', 'D-TUE', 'W-TUE', 'M-TUE']:
        print(f'{frequency}\n')
        dm.gambler(data, frequency, 'abs_change')

In [ ]:
# Rolling average of absolute change to support gambler function
analysed_df = dm.analyser(finnifty_data, frequency='W-tue')
rolling_periods = 9
analysed_df['rolling'] = analysed_df['abs_change'].rolling(rolling_periods, min_periods=1).mean()
px.line(analysed_df, x=analysed_df.index, y='rolling')

In [ ]:
analysed_df

# One min data

In [ ]:
kite_obj = dm.get_greenlit_kite(kite_api_key, kite_api_secret, kite_user, kite_pass, kite_auth_key)

In [ ]:
# Updating one min data for NIFTY 50, NIFTY BANK and NIFTY FIN SERVICE
dm.get_1m_data(kite_obj, 'NIFTY 50', path='data\\')
dm.get_1m_data(kite_obj, 'NIFTY BANK', path='data\\')
dm.get_1m_data(kite_obj, 'NIFTY FIN SERVICE', path='data\\')

In [ ]:
nifty_ticker, nifty_weights = vs.get_index_constituents('NIFTY')

In [ ]:
for ticker in nifty_ticker:
    dm.get_1m_data(kite_obj, ticker, path='data\\')

In [ ]:
nifty_onemin = pd.read_csv('data/NIFTY 50_onemin_prices.csv', index_col=0, parse_dates=True)
bnf_onemin = pd.read_csv('data/NIFTY BANK_onemin_prices.csv', index_col=0, parse_dates=True)
fin_onemin = pd.read_csv('data/NIFTY FIN SERVICE_onemin_prices.csv', index_col=0, parse_dates=True)

# Intraday Trend

In [ ]:
trend_nifty = dm.backtest_intraday_trend(nifty_onemin, eod_client=client)
trend_bnf = dm.backtest_intraday_trend(bnf_onemin, beta = 1.35, eod_client=client)
trend_finnifty = dm.backtest_intraday_trend(fin_onemin, beta = 1.30, eod_client=client)

trend_nifty_summary = dm.daywise_returns_of_intraday_trend(trend_nifty)
trend_bnf_summary = dm.daywise_returns_of_intraday_trend(trend_bnf)
trend_finnifty_summary = dm.daywise_returns_of_intraday_trend(trend_finnifty)

trend_nifty_drivers = dm.calculate_intraday_return_drivers(nifty_onemin, trend_nifty_summary)
trend_bnf_drivers = dm.calculate_intraday_return_drivers(bnf_onemin, trend_bnf_summary)
trend_finnifty_drivers = dm.calculate_intraday_return_drivers(fin_onemin, trend_finnifty_summary)

In [ ]:
trend_bnf_drivers

In [ ]:
# Year wise summary of returns
df_to_sum = trend_nifty_drivers
df_to_sum.groupby(df_to_sum.index.year).returns.sum()

In [ ]:
all_indices_returns = trend_bnf_summary.merge(trend_finnifty_summary, left_index=True, right_index=True, suffixes=('_bnf', '_finnifty')).merge(trend_nifty_summary, left_index=True, right_index=True, suffixes=('', '_nifty'))[['returns', 'returns_bnf', 'returns_finnifty']]
all_indices_returns

In [ ]:
# Plotting the distribution of the ratio for stop loss and no stop loss days for all indices
all_indices_vertical = pd.concat([trend_bnf_drivers, trend_finnifty_drivers, trend_nifty_drivers])
all_indices_vertical_stop_loss =  all_indices_vertical[(all_indices_vertical.returns <= -0.3)]
all_indices_vertical_no_stop_loss =  all_indices_vertical[(all_indices_vertical.returns > -0.3)]
fig = px.histogram(all_indices_vertical_stop_loss, x='ratio')
fig.add_trace(go.Histogram(x=all_indices_vertical_no_stop_loss.ratio, name='No Stop Loss'))
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.75)

In [ ]:
# Plotting the minute vol and open to close trend on different y axis
df_to_plot = trend_nifty_drivers
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Line(x=df_to_plot.index, y=df_to_plot['rolling_ratio'], name='Ratio'), secondary_y=True)
fig.add_trace(go.Line(x=df_to_plot.index, y=df_to_plot['returns_rolling'], name='Returns'), secondary_y=False)

# Modelling IV surface

In [ ]:
# Modelling IV surface
vol_surface = pd.read_csv('data/vol_surface.csv', index_col=0)
#vol_surface = vol_surface.drop(vol_surface[vol_surface.isna().all(axis=1)].index)
vol_surface['tte'] = vol_surface.time_to_expiry.apply(lambda num: round(num, 4))
vol_surface

In [ ]:
# Modelling IV surface
vol_surface_dict = {}
for tte in vol_surface.tte.unique():
    X = vol_surface.loc[vol_surface.tte == tte][['distance', 'distance_squared']]
    y = vol_surface.loc[vol_surface.tte == tte]['iv_multiple']
    model = LinearRegression()
    model.fit(X, y)
    dis_sq_coeff, dis_coeff, intercept = model.coef_[1], model.coef_[0], model.intercept_
    score = model.score(X, y)
    if score > 0.9:
        vol_surface_dict[tte] = {'dis_sq_coeff': dis_sq_coeff, 'dis_coeff': dis_coeff, 'intercept': intercept, 'score': score}
    # print(f'{tte} days to expiry: Coefficients: {model.coef_}, Intercept: {model.intercept_}, R2: {model.score(X, y)}')
vol_surface_weights = pd.DataFrame(vol_surface_dict).T.reset_index().rename(columns={'index': 'time_to_expiry'})
vol_surface_weights.sort_values('time_to_expiry', inplace=True)

In [ ]:
vol_surface_weights

In [ ]:
fig = px.scatter(vol_surface_weights, x='time_to_expiry', y='dis_sq_coeff')
fig.show()

In [ ]:
def func(x, a, b, c):
    return a * np.exp(-b * x) + c
lower_bounds = [-np.inf, -np.inf, -np.inf]
upper_bounds = [np.inf, np.inf, np.inf]
popt, pcov = curve_fit(func, vol_surface_weights['time_to_expiry'], vol_surface_weights['dis_sq_coeff'], bounds=(lower_bounds, upper_bounds))

In [ ]:
dummy_range = np.arange(0, 1, 0.0001)
fig.add_trace(px.line(x=dummy_range, y=func(dummy_range, *popt)).data[0])

In [ ]:
popt

In [ ]:
# Modelling IV surface - Distance Squared coefficient vs Time to Expiry (inverse)
for param in np.arange(0.02, 1.5, 0.01):
    vol_surface_weights['tte_inverse'] = 1 / (vol_surface_weights.time_to_expiry**param)
    X = vol_surface_weights['tte_inverse'].values.reshape(-1, 1)
    y = vol_surface_weights['dis_coeff']
    model = LinearRegression()
    model.fit(X, y)
    print(f'{param} param: Coefficients: {model.coef_}, Intercept: {model.intercept_}, R2: {model.score(X, y)}')

In [ ]:
px.scatter(vol_surface_weights, x='time_to_expiry', y='dis_coeff')

In [ ]:
def coefficients_for_surface(tte):

    # distance squared coefficient
    dfs2 = 3270.27*np.exp(-384.38*tte) + 100
    dfs2 = min(dfs2, 20000)

    # distance coefficient
    if tte < 0.26/365:
        dfs = 1
    else:
        dfs = 1 / ((tte ** 0.45) * 5)
        dfs = min(dfs, 5)
        dfs = -6 + dfs

    # intercept
    if tte<3/(24*365):
        intercept=1.07
    elif tte<0.27/365:
        intercept=1
    else:
        intercept=0.98
    return dfs2, dfs, intercept

In [ ]:
coefficients_for_surface(2/365)